In [3]:
import pandas as pd
import numpy as np
import csv
from Bio import SeqIO
import os

In [21]:
def find_matches(peptide, db):
    search = {}
    for string in peptide:
        search[string] = []
        for key, value in db.items():
            if string in value:
                    search[string].append((key))
    
    unmatched = []
    #fn for filtering the 6FT matched dictionary
    def my_filtering_function(pair):
        key, value = pair
        if value == []:
            return False  # filter pair out of the dictionary
        else:
            return True  # keep pair in the filtered dictionary
        
    matched = dict(filter(my_filtering_function, search.items()))

    for key, value in search.items():
        if (value == []):
            unmatched.append(key)
    if (len(matched) + len(unmatched) == len(search)):  #to make sure the dictionary filter works fine
        return matched
    else:
        return print("\t" + "error in separating matches")
    

def split_string(s):
    splits_dict = {}
    # Ensure the string is long enough to be split into two parts each with at least `min_length` characters
    if len(s) < 6:
        return splits_dict
    
    for i in range(3, len(s) - 2):
        part1 = s[:i]
        part2 = s[i:]
        if len(part2) >= 3:
            splits_dict[part1] = part2
    return splits_dict

def PCPS(input_file):
        
        input1= SeqIO.parse('db/human_canonical.fasta',"fasta") 
        seqdb={}
        for record in input1:
                seq=str(record.seq)
                if record.id not in seqdb:
                        seqdb[record.id]=seq

        input = open(input_file, "r")
        output = open('cis_PCPS_out', "w")
        output_2 = open('trans_PCPS_out', "w")
        for line in input:
                row=line.strip().split("\t")
                pep = row[0]
                split_combinations = split_string(pep)
                part1_list = list(split_combinations.keys())
                part2_list = list(split_combinations.values())
                match_1 = find_matches(part1_list, seqdb)
                match_2 = find_matches(part2_list, seqdb)
                row = []
                for key,value in match_1.items():
                        for pro_1 in match_1[key]:
                                if (split_combinations[key] in match_2.keys()):
                                        for pro_2 in match_2[split_combinations[key]]:
                                                if pro_1 == pro_2:
                                                    result = (f"{key}|{split_combinations[key]}" + "\t" + f"{pro_2}")
                                                    output.write(str(pep) + "\t" + result + "\n")
                                                else:
                                                    result_2 = (f"{key}|{split_combinations[key]}" + "\t" + f"{pro_1}:{pro_2}")
                                                    output_2.write(str(pep) + "\t" + result_2 + "\n")
        input.close()
        output.close()
        output_2.close()



In [25]:
#fn that gets matches and position of matches from peptide_list to fasta

def get_pos(substring, string):
# Find the starting position
    start_position = string.find(substring)

    if start_position != -1:
        # Calculate the ending position
        end_position = start_position + len(substring) - 1
        return(start_position, end_position)
    

def find_matches(peptide, db):
    search_6ft = {}
    pos_6ft = {}
    for string in peptide:
        search_6ft[string] = []
        pos_6ft[string] = []
        for key, value in db.items():
            if string in value:
                    search_6ft[string].append((key))
                    pos_6ft[string].append(get_pos(string,value))
                        
    unmatched = []
    #fn for filtering the 6FT matched dictionary
    def my_filtering_function(pair):
        key, value = pair
        if value == []:
            return False  # filter pair out of the dictionary
        else:
            return True  # keep pair in the filtered dictionary
        
    matched = dict(filter(my_filtering_function, search_6ft.items()))
    positions = dict(filter(my_filtering_function, pos_6ft.items()))

    for key, value in search_6ft.items():
        if (value == []):
            unmatched.append(key)
    if (len(matched) + len(unmatched) == len(search_6ft)):  #to make sure the dictionary filter works fine
        return (matched, unmatched, positions)
    else:
        return print("\t" + "error in separating matches")
    

def split_string(s):
    splits_dict = {}
    # Ensure the string is long enough to be split into two parts each with at least `min_length` characters
    if len(s) < 6:
        return splits_dict
    
    for i in range(3, len(s) - 2):
        part1 = s[:i]
        part2 = s[i:]
        if len(part2) >= 3:
            splits_dict[part1] = part2
    return splits_dict

def PCPS(input_file):
        
        input1= SeqIO.parse('db/human_canonical.fasta',"fasta") 
        seqdb={}
        for record in input1:
                seq=str(record.seq)
                if record.id not in seqdb:
                        seqdb[record.id]=seq

        input = open(input_file, "r")
        output = open('cis_PCPS_out', "w")
        output_2 = open('trans_PCPS_out', "w")
        for line in input:
                row=line.strip().split("\t")
                pep = row[0]
                split_combinations = split_string(pep)
                part1_list = list(split_combinations.keys())
                part2_list = list(split_combinations.values())
                match_1, unmatch_1, pos_1 = find_matches(part1_list, seqdb)
                match_2, unmatch_2, pos_2 = find_matches(part2_list, seqdb)
                row = []
                for key,value in match_1.items():
                        for pro_1 in match_1[key]:
                                if (split_combinations[key] in match_2.keys()):
                                        for pro_2 in match_2[split_combinations[key]]:
                                                if pro_1 == pro_2:
                                                    result = (f"{key}|{split_combinations[key]}" + "\t" + f"{pro_2}")
                                                    output.write(str(pep) + "\t" + result + "\n")
                                                else:
                                                    result_2 = (f"{key}|{split_combinations[key]}" + "\t" + f"{pro_1}:{pro_2}")
                                                    output_2.write(str(pep) + "\t" + result_2 + "\n")
        input.close()
        output.close()
        output_2.close()



In [13]:
pwd

'/Users/rishyashrung/Proteomics/Immunosearch/test'

In [12]:
cd test

[Errno 2] No such file or directory: 'test'
/Users/rishyashrung/Proteomics/Immunosearch/test


In [26]:
PCPS("no_match_6ft_2.fasta")
# PCPS = pd.read_table("trans_PCPS_out", names= ["Peptide", "PCPS_match"])
# spliced_peps = pd.unique (PCPS["Peptide"])
# from_6ft = pd.read_table("no_match_6ft_2.fasta", names = ["Peptides"])
# remaining = from_6ft[~from_6ft["Peptides"].isin(spliced_peps)]